## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
from pymystem3 import Mystem
import pandas as pd
import collections

In [2]:
# data = pd.read_csv('data.csv') #nrows = 5
data = pd.read_csv('/datasets/data.csv') # код ревьюера
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


In [4]:
data.groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
безработный,2,2,2,2,2,2,2,2,2,2,2
в декрете,1,1,1,1,1,1,1,1,1,1,1
госслужащий,1459,1312,1459,1459,1459,1459,1459,1459,1459,1312,1459
компаньон,5085,4577,5085,5085,5085,5085,5085,5085,5085,4577,5085
пенсионер,3856,3443,3856,3856,3856,3856,3856,3856,3856,3443,3856
предприниматель,2,1,2,2,2,2,2,2,2,1,2
сотрудник,11119,10014,11119,11119,11119,11119,11119,11119,11119,10014,11119
студент,1,1,1,1,1,1,1,1,1,1,1


In [5]:
data.groupby('family_status').count()

,children,days_employed,dob_years,education,education_id,family_status_id,gender,income_type,debt,total_income,purpose
family_status,,,,,,,,,,,
Не женат / не замужем,2813,2525,2813,2813,2813,2813,2813,2813,2813,2525,2813
в разводе,1195,1083,1195,1195,1195,1195,1195,1195,1195,1083,1195
вдовец / вдова,960,865,960,960,960,960,960,960,960,865,960
гражданский брак,4177,3735,4177,4177,4177,4177,4177,4177,4177,3735,4177
женат / замужем,12380,11143,12380,12380,12380,12380,12380,12380,12380,11143,12380


In [6]:
data.groupby('purpose').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income
purpose,,,,,,,,,,,
автомобили,478,421,478,478,478,478,478,478,478,478,421
автомобиль,495,454,495,495,495,495,495,495,495,495,454
высшее образование,453,413,453,453,453,453,453,453,453,453,413
дополнительное образование,462,414,462,462,462,462,462,462,462,462,414
жилье,647,587,647,647,647,647,647,647,647,647,587
заняться высшим образованием,496,440,496,496,496,496,496,496,496,496,440
заняться образованием,412,357,412,412,412,412,412,412,412,412,357
на покупку автомобиля,472,442,472,472,472,472,472,472,472,472,442
на покупку подержанного автомобиля,479,437,479,479,479,479,479,479,479,479,437


In [7]:
data.groupby('debt').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,total_income,purpose
debt,,,,,,,,,,,
0,19784,17780,19784,19784,19784,19784,19784,19784,19784,17780,19784
1,1741,1571,1741,1741,1741,1741,1741,1741,1741,1571,1741


In [8]:
data.groupby('gender').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,income_type,debt,total_income,purpose
gender,,,,,,,,,,,
F,14236,12752,14236,14236,14236,14236,14236,14236,14236,12752,14236
M,7288,6598,7288,7288,7288,7288,7288,7288,7288,6598,7288
XNA,1,1,1,1,1,1,1,1,1,1,1


In [9]:
data.dob_years.count()

21525

### Вывод

- Пропущены значения в **days_employed** и **total_income**, нужно будет обработать.
- **education** - привести к одному формату
- Поменять тип **days_employed** на целочисленный.
- **debt** поменять тип на логический
- **total_income** - поменять тип на целочисленный
- Леммаризовать **purpose**

### Шаг 2. Предобработка данных

### Обработка пропусков

Заполним отсутствующие значения **days_employed** медианой. Поскольку отрицательных дней не бывает, меняем знак где надо. Медиана выбрана потому что есть явно завышенные значения, и среднее арифметическое с учетом этого будет завышено тоже. Для **total_income** тоже выбираем медиану, поскольку средний доход достовернее показывать именно этим характерным значением.

Причины пропусков не очень понятны возможно часть данных о работе и доходе просто потерялись где то.

In [10]:
data.days_employed = data.days_employed.abs() # меняем знак
data.days_employed.median() # смотрим на медиану

2194.220566878695

In [11]:
data.days_employed.isna().sum() # количество пропущеных значений

2174

In [12]:
data.days_employed = data.days_employed.fillna(data.days_employed.median()) # заполняем пропущенные значения
data.days_employed.isna().sum() 

0

In [13]:
data.total_income.median()

145017.93753253992

In [14]:
data.total_income.isna().sum()

2174

In [15]:
data.total_income = data.total_income.fillna(data.total_income.median())

In [16]:
data.total_income.isna().sum()

0

Еще мы нашли ошибочные значения которые тоже было бы неплохо обработать:
- явно завышенные (раз так в 100-200 значения в стоблце **days_employed**
- завышенное количество семей у которых 20 детей, что скорее всего тоже ошибка
- заемщик с неправильным **gender**

Для замены ошибочных значений **days_employed**, поскольку большинство заемщиков с ошибочными данными это пенсионеры, можно выставить в столбец отработанных дней значение соответствующее 30-40 годам, для семей с 20 детьми - поставим 2 детей, так как скорее всего ошибка возникла из-за добавления нуля. Заемщика с неправильным **gender** удалим, потому что он единственный и его удаление значимо не повлияет на исследование данных.

In [17]:
data['employed_year'] = (data.days_employed/365).astype('int64')
data[data['employed_year']>(data['dob_years'])].groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose,employed_year
income_type,,,,,,,,,,,,
безработный,2,2,2,2,2,2,2,2,2,2,2,2
госслужащий,6,6,6,6,6,6,6,6,6,6,6,6
компаньон,18,18,18,18,18,18,18,18,18,18,18,18
пенсионер,3446,3446,3446,3446,3446,3446,3446,3446,3446,3446,3446,3446
сотрудник,49,49,49,49,49,49,49,49,49,49,49,49


In [18]:
data[data['employed_year']>(data['dob_years'])].income_type.value_counts()

пенсионер      3446
сотрудник        49
компаньон        18
госслужащий       6
безработный       2
Name: income_type, dtype: int64

In [19]:
data.days_employed.median()

2194.220566878695

In [20]:
data.days_employed.mean()

60378.0327334873

In [21]:
data.loc[data['days_employed'] > data['dob_years']*365, 'days_employed'] = 35*365

Прикинул что в среднем пенсионер проработает лет 35 до пенсии (может и больше 

In [22]:
data.days_employed.median()

2194.220566878695

In [23]:
data.days_employed.mean()

4046.2491016855333

In [24]:
data.loc[data['children'] == 20, 'children'] = 2 # меняем 20 на 2

In [25]:
data.groupby('children').count()

,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_year
children,,,,,,,,,,,,
-1,47,47,47,47,47,47,47,47,47,47,47,47
0,14149,14149,14149,14149,14149,14149,14149,14149,14149,14149,14149,14149
1,4818,4818,4818,4818,4818,4818,4818,4818,4818,4818,4818,4818
2,2131,2131,2131,2131,2131,2131,2131,2131,2131,2131,2131,2131
3,330,330,330,330,330,330,330,330,330,330,330,330
4,41,41,41,41,41,41,41,41,41,41,41,41
5,9,9,9,9,9,9,9,9,9,9,9,9


In [26]:
data = data[data['gender'] != 'XNA']
data.groupby('gender').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,income_type,debt,total_income,purpose,employed_year
gender,,,,,,,,,,,,
F,14236,14236,14236,14236,14236,14236,14236,14236,14236,14236,14236,14236
M,7288,7288,7288,7288,7288,7288,7288,7288,7288,7288,7288,7288


### Вывод

Обработали пропуски характерным значением (медианой). Обработали ошибочные значения. 

### Замена типа данных

**total_income** и **days_employed** меняем на целочисленный, **debt** меняем на логический.

In [27]:
data.total_income = data.total_income.astype('int64')
data.days_employed = data.days_employed.astype('int64')
data.debt = data.debt.astype('bool')

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 13 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null bool
total_income        21524 non-null int64
purpose             21524 non-null object
employed_year       21524 non-null int64
dtypes: bool(1), int64(7), object(5)
memory usage: 2.2+ MB


### Вывод

Поменяли, получилось! Для замены типа данных используется метод pandas astype(), потому что он хорошо и корректно меняет тип данных.

### Обработка дубликатов

Для того что бы обработать дубликаты приведем значения столбца **education** к одному формату.
Поменяем отричательное количество детей в столбце **children** на положительное.

Будем использовать метод duplicated() и метод sum() для поиска и подсчета количества дубликатов.
Для удаления дупликатов будем использовать метод drop_duplicates()

Причины появления дубликатов - ошибки в исходных данных, разные форматы значений **education**, **children**.

In [29]:
data.groupby('education').count()

,children,days_employed,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_year
education,,,,,,,,,,,,
ВЫСШЕЕ,274,274,274,274,274,274,274,274,274,274,274,274
Высшее,268,268,268,268,268,268,268,268,268,268,268,268
НАЧАЛЬНОЕ,17,17,17,17,17,17,17,17,17,17,17,17
НЕОКОНЧЕННОЕ ВЫСШЕЕ,29,29,29,29,29,29,29,29,29,29,29,29
Начальное,15,15,15,15,15,15,15,15,15,15,15,15
Неоконченное высшее,47,47,47,47,47,47,47,47,47,47,47,47
СРЕДНЕЕ,772,772,772,772,772,772,772,772,772,772,772,772
Среднее,711,711,711,711,711,711,711,711,711,711,711,711
УЧЕНАЯ СТЕПЕНЬ,1,1,1,1,1,1,1,1,1,1,1,1


In [30]:
data['education'] = data['education'].str.lower()
data.groupby('education').count()

,children,days_employed,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_year
education,,,,,,,,,,,,
высшее,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260
начальное,282,282,282,282,282,282,282,282,282,282,282,282
неоконченное высшее,743,743,743,743,743,743,743,743,743,743,743,743
среднее,15233,15233,15233,15233,15233,15233,15233,15233,15233,15233,15233,15233
ученая степень,6,6,6,6,6,6,6,6,6,6,6,6


In [31]:
data.children = data.children.abs()

In [32]:
data.duplicated().sum()

71

In [33]:
data = data.drop_duplicates()
data.duplicated().sum()

0

### Вывод

Мы удалили найденные дубликаты. После лемматизации нужно будет повторить процесс поиска и удалить снова.

### Лемматизация

Применим лемматизацию для сортировки раздувшегося списка целей. разобьем на следующие категории: автомобиль, образование, свадьба, и жилье. Жилье дополнительно разобьем на покупку, ремонт и стоительство.

Для применения функции к данным будем использовать метод map().

In [34]:
collections.Counter(data['purpose'])

Counter({'покупка жилья': 646,
         'приобретение автомобиля': 461,
         'дополнительное образование': 460,
         'сыграть свадьбу': 765,
         'операции с жильем': 652,
         'образование': 447,
         'на проведение свадьбы': 768,
         'покупка жилья для семьи': 638,
         'покупка недвижимости': 620,
         'покупка коммерческой недвижимости': 661,
         'покупка жилой недвижимости': 606,
         'строительство собственной недвижимости': 635,
         'недвижимость': 633,
         'строительство недвижимости': 619,
         'на покупку подержанного автомобиля': 478,
         'на покупку своего автомобиля': 505,
         'операции с коммерческой недвижимостью': 650,
         'строительство жилой недвижимости': 624,
         'жилье': 646,
         'операции со своей недвижимостью': 627,
         'автомобили': 478,
         'заняться образованием': 408,
         'сделка с подержанным автомобилем': 486,
         'получение образования': 442,
         'авт

In [35]:
def lemmat(text):

    t1 = m.lemmatize(text)
    if 'автомобиль' in t1: return 'автомобиль'
    if 'образование' in t1: return 'образование'
    if 'свадьба' in t1: return 'свадьба'
    if 'жилье' or 'недвижимость' in t1: 
        if 'покупка' in t1: return 'покупка жилья'
        elif 'ремонт' in t1: return 'ремонт жилья'
        elif 'строительство' in t1: return 'строительство жилья'
        else: return 'покупка жилья'
    

m = Mystem()
data['purpose'] = data['purpose'].map(lemmat)

In [36]:
collections.Counter(data['purpose'])

Counter({'покупка жилья': 8325,
         'автомобиль': 4306,
         'образование': 4013,
         'свадьба': 2324,
         'строительство жилья': 1878,
         'ремонт жилья': 607})

In [37]:
data.groupby('purpose').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,employed_year
purpose,,,,,,,,,,,,
автомобиль,4306,4306,4306,4306,4306,4306,4306,4306,4306,4306,4306,4306
образование,4013,4013,4013,4013,4013,4013,4013,4013,4013,4013,4013,4013
покупка жилья,8325,8325,8325,8325,8325,8325,8325,8325,8325,8325,8325,8325
ремонт жилья,607,607,607,607,607,607,607,607,607,607,607,607
свадьба,2324,2324,2324,2324,2324,2324,2324,2324,2324,2324,2324,2324
строительство жилья,1878,1878,1878,1878,1878,1878,1878,1878,1878,1878,1878,1878


In [38]:
data.duplicated().sum()

254

In [39]:
data = data.drop_duplicates()
data.duplicated().sum()

0

### Вывод

Лемматизация удалась. Дубликаты появившиеся из-за нее, почищены!

### Категоризация данных

Для последующих вопросов, разобьем заемщиков по категориям по их доходу. Для вопроса про детей категоризуем по количеству детей. 

In [40]:
def inc_group(income):
    if income <= 80000:
        return 'до 80к'
    elif 80001 <= income <= 100000:
        return 'от 80к до 100к'
    elif 100001 <= income <= 120000:
        return 'от 100к до 120к'
    elif 120001 <= income <= 130000:
        return 'от 120к до 130к'
    elif 130001 <= income <= 140000:
        return 'от 130к до 140к'
    elif 140001 <= income <= 150000:
        return 'от 140к до 150к'
    elif 150001 <= income <= 180000:
        return 'от 150к до 180к'
    elif 180001 <= income <= 200000:
        return 'от 180к до 200к'
    elif 200001 <= income <= 250000:
        return 'от 200к до 250к'
    else:
        return 'от 250к'

data['incom_group'] = data.total_income.apply(inc_group)

In [41]:
data.groupby('incom_group').debt.count().sort_values(ascending=False)

incom_group
от 140к до 150к    2911
от 250к            2812
от 150к до 180к    2762
от 100к до 120к    2382
до 80к             2276
от 200к до 250к    2253
от 80к до 100к     2187
от 180к до 200к    1356
от 130к до 140к    1162
от 120к до 130к    1098
Name: debt, dtype: int64

In [42]:
def child(children):
    if children < 1:
        return 'бездетные'
    elif children == 1:
        return 'один'
    elif children == 2:
        return 'два'
    else:
        return 'три и больше'

data['child'] = data.children.apply(child)

In [43]:
data.groupby('child').debt.count().sort_values(ascending=False)

child
бездетные       13888
один             4814
два              2118
три и больше      379
Name: debt, dtype: int64

### Вывод

Категоризовали данные по доходу и количеству детей. Пригодится нам для анализа зависимостей в следующем шаге.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [44]:
data.groupby('child').debt.count().sort_values(ascending=False)

child
бездетные       13888
один             4814
два              2118
три и больше      379
Name: debt, dtype: int64

In [45]:
(data.groupby('child').debt.sum() / data.groupby('child').debt.count()).sort_values(ascending=False)

child
два             0.095373
один            0.092439
три и больше    0.081794
бездетные       0.076469
Name: debt, dtype: float64

In [46]:
(data.pivot_table(index='child', values='debt', aggfunc='sum')/data.pivot_table(index='child', values='debt', aggfunc='count')).sort_values('debt',ascending=False).reset_index()

,child,debt
0,два,0.095373
1,один,0.092439
2,три и больше,0.081794
3,бездетные,0.076469


### Вывод

Чаще всего возвращают кредиты в срок **бездетные** заемщики, хуще всех у кого **один** или **два** ребенка. Про **многодетные** семьи толком ничего сказать не можем, потому что их на порядок меньше, но по тем данным что у нас есть, возвращают кредиты они хуже чем **бездетные** но лучше чем заемщики с **одним** или **двумя** детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [48]:
(data.groupby('family_status').debt.sum() / data.groupby('family_status').debt.count()).sort_values(ascending=False)

family_status
Не женат / не замужем    0.098173
гражданский брак         0.094083
женат / замужем          0.076587
в разводе                0.071249
вдовец / вдова           0.066456
Name: debt, dtype: float64

In [49]:
(data.pivot_table(index='family_status', values='debt', aggfunc='sum')/data.pivot_table(index='family_status', values='debt', aggfunc='count')).sort_values('debt',ascending=False).reset_index()

,family_status,debt
0,Не женат / не замужем,0.098173
1,гражданский брак,0.094083
2,женат / замужем,0.076587
3,в разводе,0.071249
4,вдовец / вдова,0.066456


### Вывод

Не любят платить в срок **не женатые / не замужние** люди и люди живущие в **гражданском браке**. **Разведенные** и **вдовые** чаще платят в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [50]:
(data.groupby('incom_group').debt.sum() / data.groupby('incom_group').debt.count()).sort_values(ascending=False)

incom_group
от 140к до 150к    0.092065
от 150к до 180к    0.089790
от 180к до 200к    0.088496
от 120к до 130к    0.087432
от 130к до 140к    0.085198
от 100к до 120к    0.082704
от 80к до 100к     0.082305
до 80к             0.076450
от 200к до 250к    0.072792
от 250к            0.068990
Name: debt, dtype: float64

In [51]:
(data.pivot_table(index='incom_group', values='debt', aggfunc='sum')/data.pivot_table(index='incom_group', values='debt', aggfunc='count')).sort_values('debt',ascending=False).reset_index()

,incom_group,debt
0,от 140к до 150к,0.092065
1,от 150к до 180к,0.089790
2,от 180к до 200к,0.088496
3,от 120к до 130к,0.087432
4,от 130к до 140к,0.085198
5,от 100к до 120к,0.082704
6,от 80к до 100к,0.082305
7,до 80к,0.076450
8,от 200к до 250к,0.072792
9,от 250к,0.068990


### Вывод

Хуже всех платят в срок люди с доходом **от 120к до 200к**. Лучше всех платят категории с доходом **до 80к** и люди с доходом **от 200к**. Остальные средненько.

- Как разные цели кредита влияют на его возврат в срок?

Найдем количество не вернувших кредит в срок и поделим его на количество по категориям. Отсортирует по убыванию.

In [52]:
(data.groupby('purpose').debt.sum() / data.groupby('purpose').debt.count()).sort_values(ascending=False)

purpose
автомобиль             0.094101
образование            0.093340
свадьба                0.080659
строительство жилья    0.076964
покупка жилья          0.073725
ремонт жилья           0.057661
Name: debt, dtype: float64

In [53]:
(data.pivot_table(index='purpose', values='debt', aggfunc='sum')/data.pivot_table(index='purpose', values='debt', aggfunc='count')).sort_values('debt',ascending=False).reset_index()

,purpose,debt
0,автомобиль,0.094101
1,образование,0.093340
2,свадьба,0.080659
3,строительство жилья,0.076964
4,покупка жилья,0.073725
5,ремонт жилья,0.057661


### Вывод

Чаще всего не вовремя возвращают кредит, взявшие его на такие цели как **автомобиль** и **образование**, чаще всего возвращают кредит в срок люди взявшие деньги на **ремонт жилья**.

### Шаг 4. Общий вывод

Что сделали: заполнили пропуски, выявили и исправили ошибки, удалили дубликаты, провели категоризацию и лемматизацию. Ответили на вопросы, зависимости своевременного возврата от количества детей, семейного положения, величины дохода и цели.

Что можно было сделать еще: проанализировать зависимость своевременного возврата от возраста, отработанного времени и пола, убрать дублирующие столбцы с образованием и статуса семейного положения, добавить коммерческую недвижимость в цели кредита.

Ну и вывод по цели проекта: будем давать кредиты разведенным заемщикам без детей с уровнем дохода либо больше 250к либо меньше 80к на ремонт или покупку жилья. Не будем давать кредиты не женатым / не замужним заемщикам с детьми с уровнем дохода от 120к до 200к на автомобиль или образование.